In [1304]:
from linopy import *
import pandas as pd
import xarray as xr
from linopy import Model
from matplotlib import cm
import numpy as np
import warnings
warnings.filterwarnings("ignore")

### Data
- import data
- print the sheet names without looking at the file

In [1305]:

data = pd.ExcelFile("./OverviewM2.xlsx")
data.sheet_names

['Simulation',
 'Input_powerplants',
 'Input_demand',
 'Input_RES',
 'Input_RES_2',
 'Input_Storage',
 'Input_Grid']

### Create a data set
- select the sheet name that contains the necessary data
- print the dataframe
- select the correct range of the data
  

In [1306]:
inpp = pd.read_excel(data, sheet_name='Input_powerplants', skiprows=1) # (inpp - Input Power Plants)

In [1307]:
col1 = inpp.iloc[1, 0:2]
col2 = inpp.iloc[0,2:]
inpp = inpp.iloc[2:9, 0:]
inpp.columns = pd.concat([col1, col2], axis=0, ignore_index=True)
inpp = inpp.loc[inpp.index.repeat(inpp['N_UNITS'])].drop(['N_UNITS'], axis =1)
inpp['Count'] = inpp.groupby('Technology').cumcount() + 1
inpp['Technology'] = inpp['Technology'] + inpp['Count'].astype(str)
inpp.set_index('Technology', inplace=True)

In [1308]:
inpp 


,Fuel,Node,GEN_MAX,EFF,GEN_MIN,EFF_PMIN,FUEL_COST,CO2_INT,DELTA_MAX_UP,DELTA_MAX_DOWN,...,MINDOWN,STC2,STC_IND,RC,STC,C,MA,D,MB,Count
Technology,,,,,,,,,,,,,,,,,,,,,
Nuclear1,UO2,1,1200,33,600,30,2,0,450,450,...,32,35,0,0,0,4000,5.454545,0,0,1
Nuclear2,UO2,1,1200,33,600,30,2,0,450,450,...,32,35,0,0,0,4000,5.454545,0,0,2
Nuclear3,UO2,1,1200,33,600,30,2,0,450,450,...,32,35,0,0,0,4000,5.454545,0,0,3
ConventionalC1,Coal,1,800,40,344,37.299296,12,0.3384,240,240,...,12,25,55,1.8,0,11067.233078,28.361331,312.095948,0.799789,1
ConventionalC2,Coal,1,800,40,344,37.299296,12,0.3384,240,240,...,12,25,55,1.8,0,11067.233078,28.361331,312.095948,0.799789,2
ConventionalC3,Coal,1,800,40,344,37.299296,12,0.3384,240,240,...,12,25,55,1.8,0,11067.233078,28.361331,312.095948,0.799789,3
ConventionalC4,Coal,1,800,40,344,37.299296,12,0.3384,240,240,...,12,25,55,1.8,0,11067.233078,28.361331,312.095948,0.799789,4
CCGT1,Gas,1,450,55,157.5,43.451957,25,0.2052,405,405,...,4,5,40,0.5,0,9061.732325,38.949788,74.378693,0.534945,1
CCGT2,Gas,1,450,55,157.5,43.451957,25,0.2052,405,405,...,4,5,40,0.5,0,9061.732325,38.949788,74.378693,0.534945,2


In [1309]:
inpp =inpp.to_xarray()    #  inpp = inpp.set_coords(['Technology', 'Fuel'])
inpp

<xarray.Dataset>
Dimensions:         (Technology: 23)
Coordinates:
  * Technology      (Technology) object 'Nuclear1' 'Nuclear2' ... 'GT4' 'GT5'
Data variables: (12/21)
    Fuel            (Technology) object 'UO2' 'UO2' 'UO2' ... 'Gas' 'Gas' 'Gas'
    Node            (Technology) object 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    GEN_MAX         (Technology) object 1200 1200 1200 800 800 ... 50 50 50 50
    EFF             (Technology) object 33 33 33 40 40 40 ... 55 40 40 40 40 40
    GEN_MIN         (Technology) object 600 600 600 344 344 ... 15 15 15 15 15
    EFF_PMIN        (Technology) object 30 30 ... 25.55232634808877
    ...              ...
    STC             (Technology) object 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    C               (Technology) object 4000 4000 ... 1467.5767477744691
    MA              (Technology) object 5.454545454545455 ... 47.35495006358659
    D               (Technology) object 0 0 ... 12.045868982063324
    MB              (Technology) object 0 0 ... 0.38868939902676675
    Count           (Technology) int64 1 2 3 1 2 3 4 1 2 ... 8 9 10 11 1 2 3 4 5

### Data Demand

In [1310]:
inpd = pd.read_excel(data, sheet_name='Input_demand', skiprows=3)
inpd = inpd.set_index(['day', 'month', 'year'])
inpd

,,,0:15,0:30,0:45,1:00,1:15,1:30,1:45,2:00,2:15,2:30,...,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45,24:00
day,month,year,,,,,,,,,,,,,,,,,,,,,
1,6.0,2013.0,8673.042,8429.322,8308.632,8140.671,8006.779,7807.729,7763.125,7634.726,7587.030,7482.310,...,7982.102,7988.921,8209.316,8370.824,8393.274,8380.822,8503.550,8353.015,8281.022,8177.031
2,6.0,2013.0,8052.679,7856.342,7743.303,7650.331,7528.002,7468.452,7255.969,7101.194,7034.424,7089.714,...,7739.691,7767.356,7966.283,8354.370,8369.891,8299.814,8297.342,8101.211,7764.114,7706.652
3,6.0,2013.0,7550.928,7438.698,7289.953,7176.891,7165.338,7130.795,6999.951,6880.288,6834.142,6766.567,...,8808.267,8773.263,9024.721,9411.377,9328.347,9108.317,9205.300,9041.506,8697.893,8637.049
4,6.0,2013.0,8496.884,8319.899,8196.701,7967.601,8032.783,7950.501,7928.243,7805.097,7665.461,7584.409,...,8881.317,8911.268,9190.812,9582.027,9560.055,9529.220,9546.534,9463.755,9171.279,8992.793
5,6.0,2013.0,8780.559,8687.634,8394.841,8200.335,8207.454,8095.371,7935.834,7898.115,7727.304,7793.651,...,8949.598,8914.767,9288.891,9542.090,9488.956,9440.409,9344.824,9135.563,8819.755,8710.935
6,6.0,2013.0,8530.443,8441.718,8199.603,8210.283,8023.118,7965.610,7853.734,7718.865,7712.707,7557.747,...,8738.274,8642.352,8920.258,9259.679,9206.432,9132.940,8948.530,8916.491,8732.527,8615.033
7,6.0,2013.0,8429.832,8271.792,8121.208,8134.763,8045.496,7884.031,7740.750,7558.598,7679.048,7523.200,...,8723.628,8692.391,8832.275,9265.558,9255.529,9182.997,9111.409,9003.198,8871.769,8587.459
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nodal division of demand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1311]:
demand = inpd.loc[(1,6,2013)].values.flatten().tolist() + inpd.loc[(2,6,2013), '0:15': '12:00'].values.flatten().tolist()
steps = pd.Index(range(len(demand)), name = 'steps')


In [1312]:
demand

[8673.042,
 8429.322,
 8308.632,
 8140.671,
 8006.779,
 7807.729,
 7763.125,
 7634.726,
 7587.03,
 7482.31,
 7280.466,
 7328.33,
 7235.253,
 7255.845,
 7115.311,
 7117.699,
 7164.986,
 7147.184,
 7076.854,
 7055.577,
 7111.024,
 7066.214,
 6989.871,
 7013.425,
 7062.822,
 7065.767,
 7190.701,
 7280.649,
 7312.226,
 7344.341,
 7422.505,
 7543.435,
 7718.39,
 7922.254,
 8029.958,
 8048.54,
 8130.725,
 8186.985,
 8282.242,
 8282.319,
 8376.805,
 8302.3,
 8280.987,
 8189.713,
 8173.417,
 8197.875,
 8125.143,
 8052.584,
 7957.04,
 7801.191,
 7641.423,
 7579.518,
 7442.655,
 7353.559,
 7209.948,
 7122.568,
 7074.798,
 6981.103,
 6959.751,
 6958.078,
 6913.528,
 6896.868,
 6984.256,
 7045.556,
 7118.526,
 7212.053,
 7266.338,
 7416.421,
 7480.078,
 7597.446,
 7795.905,
 7927.544,
 7867.907,
 8004.53,
 8101.022,
 8121.424,
 8158.91,
 8123.518,
 8162.692,
 8165.624,
 8178.655,
 8132.056,
 8048.951,
 7927.754,
 7971.499,
 7980.596,
 7982.102,
 7988.921,
 8209.316,
 8370.824,
 8393.274,
 8380.822

In [1313]:
RE = pd.read_excel(data, sheet_name='Input_RES_2')
RE.set_index(steps, inplace=True)
GSOLAR = RE['GSOLAR'].fillna(0)
GWIND = RE['GWIND'].fillna(0)

### Storage

In [1314]:
inps = pd.read_excel(data, sheet_name='Input_Storage', skiprows=2)
inps.drop(inps.index[0], inplace=True)
inps.rename(columns={'Unnamed: 0': 'Storage'}, inplace=True)
inps.set_index('Storage', inplace=True)
inps

,Node,Pmax,Rated efficiency (pumping),Rated efficiency (turbining),Emax,Emin
Storage,,,,,,
1,3,600,80,80,6000,600
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN
Number of storage units,1,NaN,NaN,NaN,NaN,NaN


In [1315]:
inps_set = inps.head(1).to_xarray()
inps_set

<xarray.Dataset>
Dimensions:                       (Storage: 1)
Coordinates:
  * Storage                       (Storage) object 1
Data variables:
    Node                          (Storage) object 3
    Pmax                          (Storage) object 600
    Rated efficiency (pumping)    (Storage) object 80
    Rated efficiency (turbining)  (Storage) object 80
    Emax                          (Storage) object 6000
    Emin                          (Storage) object 600

In [1316]:
gp = pd.read_excel(data, sheet_name='Simulation', skiprows=1)
gp = gp.drop(gp.columns[1], axis =1).set_index('Simulation_parameters').to_xarray()

In [1317]:
gp

<xarray.Dataset>
Dimensions:                (Simulation_parameters: 20)
Coordinates:
  * Simulation_parameters  (Simulation_parameters) object 'Session' ... 'rins...
Data variables:
    Value                  (Simulation_parameters) object 'Basic' 1 ... 0 100
    Unnamed: 3             (Simulation_parameters) object "Session: 'Basic','...

In [1318]:
gp.sel(Simulation_parameters = 'Time step') 
gp.loc[dict(Simulation_parameters = 'Time step')]
gp['Value'].sel(Simulation_parameters = 'Time step')
gp.Value.loc['Time step']

<xarray.DataArray 'Value' ()>
array(0.25, dtype=object)
Coordinates:
    Simulation_parameters  <U9 'Time step'

In [1319]:
gp


<xarray.Dataset>
Dimensions:                (Simulation_parameters: 20)
Coordinates:
  * Simulation_parameters  (Simulation_parameters) object 'Session' ... 'rins...
Data variables:
    Value                  (Simulation_parameters) object 'Basic' 1 ... 0 100
    Unnamed: 3             (Simulation_parameters) object "Session: 'Basic','...

In [1320]:
def model():
    m = Model()
    scu = m.add_variables(coords= [inpp.Technology], name = 'scu')
    return m.solution.to_dataframe(), m.objective.value

In [1321]:
m = Model()
# scu = m.add_variables(lower= 0, coords= [inpp.Technology], name = 'scu')

In [1322]:
scu = m.add_variables(lower= 0, coords= [inpp.Technology, steps], name = 'scu')
z = m.add_variables(binary = True, coords= [inpp.Technology, steps], name = 'z')
v = m.add_variables(binary = True, coords= [inpp.Technology, steps], name = 'v')
w = m.add_variables(binary = True, coords= [inpp.Technology, steps], name = 'w')
curtail  = m.add_variables( coords= [steps], name = 'curtail')
fuelcost = m.add_variables(lower = 0, coords= [inpp.Technology, steps], name = 'fuelcost') #fcd in gams
co2cost = m.add_variables(lower = 0, coords= [inpp.Technology, steps], name = 'co2cost') #ccd in gams
rampingcost = m.add_variables(lower = 0, coords= [inpp.Technology, steps], name = 'rampingcost') #rcd in gams
load_shedding = m.add_variables( coords= [steps], name = 'load_shedding')
powergen = m.add_variables(lower = 0, coords= [inpp.Technology, steps], name = 'powergen') #g
pump = m.add_variables(lower = 0, coords= [inps_set.Storage, steps], name = 'pump') #pumping
turbine = m.add_variables(lower = 0, coords= [inps_set.Storage, steps], name = 'turbine')
turbining_power = m.add_variables(lower = 0, coords= [inps_set.Storage, steps], name = 'turbining_power') #pdown in gams
pumping_power = m.add_variables(lower = 0, coords= [inps_set.Storage, steps], name = 'pumping_power') #pup in gams
puct = m.add_variables(lower = 0, coords= [inps_set.Storage, steps], name = 'puct') # energy content hydro storage

In [1323]:
cost_tot = (scu +gp.Value.loc['Time step']* fuelcost + gp.Value.loc['Time step']*co2cost+rampingcost).sum() \
        + gp.Value.loc['VOLL']*gp.Value.loc['Time step']*load_shedding.sum() \
        + gp.Value.loc['VOC']*gp.Value.loc['Time step']*curtail.sum()
m.add_objective(cost_tot)

In [1324]:
startup = m.add_constraints(inpp['STC']*v == scu, name = 'startup')
fuel = m.add_constraints( fuelcost >= inpp['C']*z + inpp['MA']*(powergen - inpp['GEN_MIN']*z), name = 'fuel')
co2 = m.add_constraints(co2cost >= gp.Value.loc['CO2_price']*(inpp['D']*z+inpp['MB']*(powergen-inpp['GEN_MIN']*z)), name = 'co2')

In [1325]:
m.add_constraints(rampingcost.loc[inpp['Technology'], 0] == powergen.loc[inpp['Technology'],0] - powergen.loc[inpp['Technology'],143])
m.add_constraints(rampingcost.loc[inpp['Technology'], 1:] >=inpp['RC']*(powergen.loc[inpp['Technology'], 1:]-powergen.shift(steps = 1)) - inpp['RC']*inpp['GEN_MIN']*v.loc[inpp['Technology'],1:])
m.add_constraints(rampingcost >= inpp['RC']*(powergen.shift(steps = 1) - powergen.loc[inpp['Technology'], 1:])- inpp['RC']*inpp['GEN_MIN']* w.shift( steps = -1))
m.add_constraints(powergen.sum(dims='Technology')+ GSOLAR + GWIND - curtail + load_shedding == demand)
curtail<= GSOLAR+GWIND

Constraint `con3` (steps: 144):
-------------------------------
[0]: +1 powergen[Nuclear1, 0] + 1 powergen[Nuclear2, 0] + 1 powergen[Nuclear3, 0] ... +1 powergen[GT5, 0] - 1 curtail[0] + 1 load_shedding[0]               = 6720.397436408814
[1]: +1 powergen[Nuclear1, 1] + 1 powergen[Nuclear2, 1] + 1 powergen[Nuclear3, 1] ... +1 powergen[GT5, 1] - 1 curtail[1] + 1 load_shedding[1]               = 6495.141630245461
[2]: +1 powergen[Nuclear1, 2] + 1 powergen[Nuclear2, 2] + 1 powergen[Nuclear3, 2] ... +1 powergen[GT5, 2] - 1 curtail[2] + 1 load_shedding[2]               = 6393.1774624563095
[3]: +1 powergen[Nuclear1, 3] + 1 powergen[Nuclear2, 3] + 1 powergen[Nuclear3, 3] ... +1 powergen[GT5, 3] - 1 curtail[3] + 1 load_shedding[3]               = 6243.9049177565585
[4]: +1 powergen[Nuclear1, 4] + 1 powergen[Nuclear2, 4] + 1 powergen[Nuclear3, 4] ... +1 powergen[GT5, 4] - 1 curtail[4] + 1 load_shedding[4]               = 6128.8135037886095
[5]: +1 powergen[Nuclear1, 5] + 1 powergen[Nuclear2, 

In [1333]:
rampingcost >= inpp['RC']*(powergen.shift(steps = 1) - powergen.loc[inpp['Technology'], 1:])- inpp['RC']*inpp['GEN_MIN']* w.shift( steps = -1)

Constraint (unassigned) (Technology: 23, steps: 144):
-----------------------------------------------------
[Nuclear1, 0]: +1 rampingcost[Nuclear1, 0] + 0 w[Nuclear1, 1]                                                     ≥ -0.0
[Nuclear1, 1]: +1 rampingcost[Nuclear1, 1] - 0 powergen[Nuclear1, 0] + 0 powergen[Nuclear1, 1] + 0 w[Nuclear1, 2] ≥ -0.0
[Nuclear1, 2]: +1 rampingcost[Nuclear1, 2] - 0 powergen[Nuclear1, 1] + 0 powergen[Nuclear1, 2] + 0 w[Nuclear1, 3] ≥ -0.0
[Nuclear1, 3]: +1 rampingcost[Nuclear1, 3] - 0 powergen[Nuclear1, 2] + 0 powergen[Nuclear1, 3] + 0 w[Nuclear1, 4] ≥ -0.0
[Nuclear1, 4]: +1 rampingcost[Nuclear1, 4] - 0 powergen[Nuclear1, 3] + 0 powergen[Nuclear1, 4] + 0 w[Nuclear1, 5] ≥ -0.0
[Nuclear1, 5]: +1 rampingcost[Nuclear1, 5] - 0 powergen[Nuclear1, 4] + 0 powergen[Nuclear1, 5] + 0 w[Nuclear1, 6] ≥ -0.0
[Nuclear1, 6]: +1 rampingcost[Nuclear1, 6] - 0 powergen[Nuclear1, 5] + 0 powergen[Nuclear1, 6] + 0 w[Nuclear1, 7] ≥ -0.0
		...
[GT5, 137]: +1 rampingcost[GT5, 137] - 

In [1328]:
(powergen.sum(dims='Technology')+RE['GSOLAR']+RE['GWIND'] - curtail + load_shedding == demand)

Constraint (unassigned) (steps: 144):
-------------------------------------
[0]: +1 powergen[Nuclear1, 0] + 1 powergen[Nuclear2, 0] + 1 powergen[Nuclear3, 0] ... +1 powergen[GT5, 0] - 1 curtail[0] + 1 load_shedding[0]               = 6720.397436408814
[1]: +1 powergen[Nuclear1, 1] + 1 powergen[Nuclear2, 1] + 1 powergen[Nuclear3, 1] ... +1 powergen[GT5, 1] - 1 curtail[1] + 1 load_shedding[1]               = 6495.141630245461
[2]: +1 powergen[Nuclear1, 2] + 1 powergen[Nuclear2, 2] + 1 powergen[Nuclear3, 2] ... +1 powergen[GT5, 2] - 1 curtail[2] + 1 load_shedding[2]               = 6393.1774624563095
[3]: +1 powergen[Nuclear1, 3] + 1 powergen[Nuclear2, 3] + 1 powergen[Nuclear3, 3] ... +1 powergen[GT5, 3] - 1 curtail[3] + 1 load_shedding[3]               = 6243.9049177565585
[4]: +1 powergen[Nuclear1, 4] + 1 powergen[Nuclear2, 4] + 1 powergen[Nuclear3, 4] ... +1 powergen[GT5, 4] - 1 curtail[4] + 1 load_shedding[4]               = 6128.8135037886095
[5]: +1 powergen[Nuclear1, 5] + 1 powerge

In [1329]:
powergen.sum()+RE['GSOLAR']+RE['GWIND']

KeyError: "No variable named 'steps'. Variables on the dataset include ['vars', 'coeffs', 'const']"

In [ ]:
powergen.shift(steps = 1)

Variable (Technology: 23, steps: 144) - 23 masked entries
---------------------------------------------------------
[Nuclear1, 0]: None
[Nuclear1, 1]: powergen[Nuclear1, 0] ∈ [0, inf]
[Nuclear1, 2]: powergen[Nuclear1, 1] ∈ [0, inf]
[Nuclear1, 3]: powergen[Nuclear1, 2] ∈ [0, inf]
[Nuclear1, 4]: powergen[Nuclear1, 3] ∈ [0, inf]
[Nuclear1, 5]: powergen[Nuclear1, 4] ∈ [0, inf]
[Nuclear1, 6]: powergen[Nuclear1, 5] ∈ [0, inf]
		...
[GT5, 137]: powergen[GT5, 136] ∈ [0, inf]
[GT5, 138]: powergen[GT5, 137] ∈ [0, inf]
[GT5, 139]: powergen[GT5, 138] ∈ [0, inf]
[GT5, 140]: powergen[GT5, 139] ∈ [0, inf]
[GT5, 141]: powergen[GT5, 140] ∈ [0, inf]
[GT5, 142]: powergen[GT5, 141] ∈ [0, inf]
[GT5, 143]: powergen[GT5, 142] ∈ [0, inf]

### selecting rows by loc method

In [ ]:
m

Linopy MILP model

Variables:
----------
 * scu (Technology, steps)
 * z (Technology, steps)
 * v (Technology, steps)
 * w (Technology, steps)
 * curtail (steps)
 * fuelcost (Technology, steps)
 * co2cost (Technology, steps)
 * rampingcost (Technology, steps)
 * load_shedding (steps)
 * powergen (Technology, steps)
 * pump (Storage, steps)
 * turbine (Storage, steps)
 * turbining_power (Storage, steps)
 * pumping_power (Storage, steps)
 * puct (Storage, steps)

Constraints:
------------
 * startup (Technology, steps)
 * fuel (Technology, steps)
 * co2 (Simulation_parameters, Technology, steps)
 * con0 (steps, Technology)
 * con1 (Technology, steps)
 * con2 (Technology, steps)
 * con3 (steps)

Status:
-------
initialized

In [ ]:
len(inpp.Technology)

23

In [ ]:
data.close()

In [ ]:
G

NameError: name 'G' is not defined

inpp